<h1 style="color:blue; text-align:center;">KNN</h1>


<h2>Importing Libraries</h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
%matplotlib inline

<h2>Load the Preprocessed Dataset</h2>

In [2]:
df = pd.read_csv("D:/HDSE 24.1/ML/Course Work/employee-attrition-prediction-ml/Dataset/after_preprocessing.csv")
# df = pd.read_csv("D:/employee-attrition-prediction-ml/Dataset/after_preprocessing.csv")
df.shape

(73915, 23)

<h2>Explore the Target Column</h2>

In [3]:
target_categories = df['Attrition'].unique()

In [4]:
print("Categories in the target column : ")
for category in target_categories:
    print(category)

Categories in the target column : 
1
0


<h2>Split the Dataset into Features and Target</h2>

In [5]:
x = df.drop(columns = [ 'Attrition'])
y = df['Attrition']

<h2>Split the Dataset into Training and Testing</h2>

In [6]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

<h2>Train a KNN Classifier</h2>

In [10]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [11]:
y_pred = knn.predict(x_test)

<h2>Evaluate the KNN Classifier</h2>

In [9]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4665 2306]
 [2469 5343]]


In [12]:
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TP = cm[1,1]

print("True Positives (TP) :", TP)
print("True Negatives (TN) :",TN)
print("False Positive (FP) :",FP)
print("False Negatives (FN)  :",FN)

True Positives (TP) : 5343
True Negatives (TN) : 4665
False Positive (FP) : 2306
False Negatives (FN)  : 2469


In [13]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_measure = 2 *(precision * recall) / ( precision + recall)

print("Accuracy : ", accuracy)
print("Precision : ", precision)
print("Recall : ", recall)
print("f_measure : ", f_measure)

Accuracy :  0.6769938442805926
Precision :  0.6985226827036214
Recall :  0.6839477726574501
f_measure :  0.6911583985511933


In [14]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.67      0.66      6971
           1       0.70      0.68      0.69      7812

    accuracy                           0.68     14783
   macro avg       0.68      0.68      0.68     14783
weighted avg       0.68      0.68      0.68     14783



<h2>Find the Best K Value Using Cross-Validation</h2>

In [ ]:
from sklearn.model_selection import cross_val_score

k_values = list(range(1, 30))
cv_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())


plt.plot(k_values, cv_scores)
plt.xlabel('Number of Neighbors (K)')
plt.ylabel('Cross-Validation Accuracy')
plt.title('KNN Performance for Different K Values')
plt.show()


best_k = k_values[np.argmax(cv_scores)]
print("Best K:", best_k)

<h2>Train and Evaluate KNN Classifier with the Best K Value</h2>

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors = 29)
best_knn.fit(x_train,y_train)
y_pred = best_knn.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)

TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TP = cm[1,1]

print("True Positives (TP) :", TP)
print("True Negatives (TN) :",TN)
print("False Positive (FP) :",FP)
print("False Negatives (FN)  :",FN)

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_measure = 2 *(precision * recall) / ( precision + recall)

print("Accuracy : ", accuracy)
print("Precision : ", precision)
print("Recall : ", recall)
print("f_measure : ", f_measure)

print(classification_report(y_test,y_pred))